# Save and load data
Utilize a prior and a simulator to create said dataset. Save a proportion as a training set, and part as a validation set. Then, run SBI using this static training set. An alternate way to run SBI using a simulator to generate data on the fly is demonstrated in the `train_SBI.ipynb` tutorial.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# remove top and right axis from plots
matplotlib.rcParams["axes.spines.right"] = False
matplotlib.rcParams["axes.spines.top"] = False

In [2]:
import sbi
from sbi.inference import SNPE
from sbi.inference.base import infer
from sbi.analysis import pairplot
import torch

In [3]:
from scripts.io import DataLoader, ModelLoader

In [4]:
def simulator(thetas):#, percent_errors):
    # convert to numpy array (if tensor):
    thetas = np.atleast_2d(thetas)
    # Check if the input has the correct shape
    if thetas.shape[1] != 2:
        raise ValueError("Input tensor must have shape (n, 2) where n is the number of parameter sets.")

    # Unpack the parameters
    if thetas.shape[0] == 1:
        # If there's only one set of parameters, extract them directly
        m, b = thetas[0, 0], thetas[0, 1]
    else:
        # If there are multiple sets of parameters, extract them for each row
        m, b = thetas[:, 0], thetas[:, 1]
    x = np.linspace(0, 100, 101)
    rs = np.random.RandomState()#2147483648)# 
    # I'm thinking sigma could actually be a function of x
    # if we want to get fancy down the road
    # Generate random noise (epsilon) based on a normal distribution with mean 0 and standard deviation sigma
    sigma = 5
    ε = rs.normal(loc=0, scale=sigma, size=(len(x), thetas.shape[0]))
    
    # Initialize an empty array to store the results for each set of parameters
    y = np.zeros((len(x), thetas.shape[0]))
    for i in range(thetas.shape[0]):
        m, b = thetas[i, 0], thetas[i, 1]
        y[:, i] = m * x + b + ε[:, i]
    return torch.Tensor(y.T)

In [5]:
low_bounds = torch.tensor([0, -10])
high_bounds = torch.tensor([10, 10])

prior = sbi.utils.BoxUniform(low = low_bounds, high = high_bounds)

To create the training set, sample from this prior and run it through the simulator.

In [6]:
params = prior.sample((10000,))
xs = simulator(params)
print(r'$\theta$s', params, 'xs', xs)

$\theta$s tensor([[ 3.0800, -4.7952],
        [ 1.8481,  6.3294],
        [ 4.0461,  2.8588],
        ...,
        [ 8.1919, -1.6658],
        [ 5.0935,  0.3070],
        [ 4.0631,  9.8509]]) xs tensor([[-8.5266e+00, -1.8273e+00, -1.1451e+00,  ...,  2.9787e+02,
          3.0003e+02,  3.1156e+02],
        [ 1.1926e+01,  6.5993e+00,  5.1448e+00,  ...,  1.8203e+02,
          1.8708e+02,  1.8575e+02],
        [-4.1232e-01,  1.1292e+01,  1.0386e+01,  ...,  3.9838e+02,
          4.0421e+02,  4.0578e+02],
        ...,
        [-1.4934e+00,  1.5889e+01,  1.6708e+01,  ...,  8.0203e+02,
          8.0764e+02,  8.1931e+02],
        [ 1.0923e+01,  6.6249e+00,  1.7376e+01,  ...,  5.0127e+02,
          5.0772e+02,  5.1401e+02],
        [ 9.2832e+00,  1.3263e+01,  2.0634e+01,  ...,  4.1520e+02,
          4.1219e+02,  4.0914e+02]])


In [7]:
# Save both params and xs to a .pkl file
data_to_save = {'thetas': params, 'xs': xs}
dataloader = DataLoader()
dataloader.save_data_h5('data_train',
                         data_to_save,
                         path = '../saveddata/')

Redo this with a validation set that is the same size.

In [8]:
params_valid = prior.sample((10000,))
xs_valid = simulator(params_valid)
print(r'$\theta$s', params_valid, 'xs', xs_valid)

$\theta$s tensor([[ 8.6844, -7.0421],
        [ 3.0001, -5.9178],
        [ 8.5659, -7.1641],
        ...,
        [ 4.7512, -6.2531],
        [ 8.1654, -1.4571],
        [ 1.2068,  3.5740]]) xs tensor([[-1.6683e+00,  5.4582e+00,  1.9455e+01,  ...,  8.4058e+02,
          8.4980e+02,  8.6225e+02],
        [-2.8954e+00, -1.2797e+01,  1.1486e+00,  ...,  2.9071e+02,
          2.8894e+02,  2.9775e+02],
        [-1.0352e+00, -1.2929e+00,  1.2169e+01,  ...,  8.3395e+02,
          8.3806e+02,  8.4531e+02],
        ...,
        [-3.2291e-01, -1.4686e+01,  9.0462e+00,  ...,  4.5691e+02,
          4.6361e+02,  4.6903e+02],
        [-7.1130e+00,  5.8362e+00,  8.4475e+00,  ...,  8.0142e+02,
          8.1157e+02,  8.0876e+02],
        [-4.1383e+00, -4.4619e+00,  1.4715e+01,  ...,  1.2880e+02,
          1.3755e+02,  1.1952e+02]])


In [9]:
# Save both params and xs to a .pkl file
data_to_save_valid = {'thetas': params_valid, 'xs': xs_valid}

dataloader.save_data_h5('data_validation',
                         data_to_save_valid,
                         path = '../saveddata/')

## Now load up this data and run SBI using it

In [10]:
train_h5 = dataloader.load_data_h5(
                         'data_train',
                        '../saveddata/',)

In [11]:
print(train_h5)

{'thetas': tensor([[ 3.0800, -4.7952],
        [ 1.8481,  6.3294],
        [ 4.0461,  2.8588],
        ...,
        [ 8.1919, -1.6658],
        [ 5.0935,  0.3070],
        [ 4.0631,  9.8509]]), 'xs': tensor([[-8.5266e+00, -1.8273e+00, -1.1451e+00,  ...,  2.9787e+02,
          3.0003e+02,  3.1156e+02],
        [ 1.1926e+01,  6.5993e+00,  5.1448e+00,  ...,  1.8203e+02,
          1.8708e+02,  1.8575e+02],
        [-4.1232e-01,  1.1292e+01,  1.0386e+01,  ...,  3.9838e+02,
          4.0421e+02,  4.0578e+02],
        ...,
        [-1.4934e+00,  1.5889e+01,  1.6708e+01,  ...,  8.0203e+02,
          8.0764e+02,  8.1931e+02],
        [ 1.0923e+01,  6.6249e+00,  1.7376e+01,  ...,  5.0127e+02,
          5.0772e+02,  5.1401e+02],
        [ 9.2832e+00,  1.3263e+01,  2.0634e+01,  ...,  4.1520e+02,
          4.1219e+02,  4.0914e+02]])}


In [12]:
# instantiate the neural density estimator
neural_posterior = sbi.utils.posterior_nn(model='maf')

'''
model,
                                  embedding_net=embedding_net,
                                  hidden_features=hidden_features,
                                  num_transforms=num_transforms)
'''

#from me:
#infer(simulator, prior, "SNPE", num_simulations=10000)

low_bounds = torch.tensor([0, -10])
high_bounds = torch.tensor([10, 10])

prior = sbi.utils.BoxUniform(low = low_bounds, high = high_bounds)

# setup the inference procedure with the SNPE-C procedure
inference = SNPE(prior=prior, density_estimator=neural_posterior, device="cpu")


density_estimator = inference.append_simulations(train_h5['thetas'],
                                                 train_h5['xs']).train()
posterior = inference.build_posterior(density_estimator)

 Neural network successfully converged after 104 epochs.

In [ ]:
modelloader = ModelLoader()
path = "../savedmodels/sbi/"
model_name = "sbi_linear_from_data"
modelloader.save_model_pkl(path, model_name, posterior)

In [ ]:
# generate a true dataset
theta_true = [1, 5]
y_true = simulator(theta_true)

# and visualize it
plt.clf()
plt.scatter(np.linspace(0, 100, 101),
            np.array(y_true), color = 'black')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# sample from the posterior
posterior_samples_1 = posterior.sample((10000,), x = y_true)
# that last little part is conditioning on a data value
# plot posterior samples
fig, axes = sbi.analysis.pairplot(
    posterior_samples_1, 
    labels = ['m', 'b'],
    #limits = [[0,10],[-10,10],[0,10]],
    truths = theta_true,
    figsize=(5, 5)
)
axes[0, 1].plot([theta_true[1]], [theta_true[0]], marker="o", color="red")